# Overview of this jupyter notebook

1. Loading the data
2. Optimize 1D-CNN Model for Multi-Label Classification with AutoKeras
3. Evaluation of the optimized model and saving the results

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, cross_validate
from sklearn.model_selection import cross_val_predict
import pickle
import json
import random


In [2]:
import tensorflow as tf
from tensorflow import keras
#import keras_tuner as kt
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential, clone_model, save_model, load_model
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import autokeras as ak
from tqdm.notebook import tqdm_notebook as tq
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
import os
import importlib
from library import etl_data as etl            # this module contains functions that load, visualize and save the data
from library import faps_color as fapsc        # this module contains the color combination of FAPS Institute
from library import classification_helper as ch  # this module contains helper functions for the classification task
print(importlib.reload(etl))
print(importlib.reload(fapsc))
print(importlib.reload(ch))

<module 'library.etl_data' from 'C:\\Users\\songs\\OneDrive\\Desktop\\0. MA_FAPS\\04_Code\\library\\etl_data.py'>
<module 'library.faps_color' from 'C:\\Users\\songs\\OneDrive\\Desktop\\0. MA_FAPS\\04_Code\\library\\faps_color.py'>
<module 'library.classification_helper' from 'C:\\Users\\songs\\OneDrive\\Desktop\\0. MA_FAPS\\04_Code\\library\\classification_helper.py'>


In [4]:
# GPU check

if len(tf.config.list_physical_devices('GPU')) > 0:
    print("TensorFlow version: ", tf.__version__)
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
    print("Name: ", tf.test.gpu_device_name())
else:
    print("TensorFlow version: ", tf.__version__)
    print("GPU is not available")

TensorFlow version:  2.8.0
Num GPUs Available:  1
Name:  /device:GPU:0


In [5]:
!python -V

Python 3.10.6


In [6]:
!nvidia-smi

Mon Mar 13 22:47:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.15       Driver Version: 512.15       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   55C    P0    N/A /  N/A |    235MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:36:24_Pacific_Standard_Time_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0


# 1. Loading the data

In [8]:
classes = ['0_in_ordnung',
           '1_kleine_durchgangsbohrung',
           '2_versetzte_fuegepartner',
           '3_unebene_fuegeflaeche',
           '4_span_im_gewinde',
           '5_verschmutzte_schraube',
           '6_fremdkoerper_am_gewindeende',
           '7_beschaedigte_schraube']

In [9]:
feature_train = np.load("feature_label/feature_train_mlc_v2.4.npy")
feature_train.shape

(1550, 920)

In [10]:
feature_test = np.load("feature_label/feature_test_mlc_v2.4.npy")
feature_test.shape

(750, 920)

In [11]:
label_train = np.load("feature_label/label_train_mlc_v2.4.npy")
label_train.shape

(1550, 8)

In [12]:
label_test = np.load("feature_label/label_test_mlc_v2.4.npy")
label_test.shape

(750, 8)

In [13]:
from sklearn.utils import shuffle

x_train, y_train = shuffle(feature_train, label_train, random_state=9)
x_test, y_test = shuffle(feature_test, label_test, random_state=9)

y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
y_test

array([[0, 0, 1, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [14]:
len_curve = 920
x_train = x_train.reshape(-1, len_curve, 1)
x_test = x_test.reshape(-1, len_curve, 1)

x_train.shape, x_test.shape

((1550, 920, 1), (750, 920, 1))

In [15]:
max_train = np.load("dataframes/max_train.npy")
max_train

array(2.56704)

# 2. Optimize 1D-CNN Model for Multi-Label Classification with AutoKeras

In [25]:
import autokeras as ak
ak.__version__

'1.0.20'

In [33]:
input_node = ak.Input()
conv_block = ak.ConvBlock()(input_node)
dense_block = ak.DenseBlock()(conv_block)
output_node = ak.ClassificationHead(num_classes=8, multi_label=True)(dense_block)

auto_model = ak.AutoModel(inputs=input_node, outputs=output_node, overwrite=True, max_trials=400,
                          objective="val_accuracy", loss='binary_crossentropy', metrics=["accuracy"], 
                          project_name="mlc_v2.4", directory="autokeras")

In [34]:
# use optimized batch size from manual optimization 

auto_model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, 
               callbacks=[EarlyStopping(monitor='val_loss', patience=10, verbose=1)])

Trial 160 Complete [00h 00m 36s]
val_accuracy: 0.9111111164093018

Best val_accuracy So Far: 0.9333333373069763
Total elapsed time: 02h 20m 05s
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
49/49 [==============================] - 2s 23ms/step - loss: 0.5421 - accuracy: 0.4529
Epoch 2/100
49/49 [==============================] - 1s 12ms/step - loss: 0.3761 - accuracy: 0.6006
Epoch 3/100
49/49 [==============================] - 1s 12ms/step - loss: 0.2907 - accuracy: 0.6794
Epoch 4/100
49/49 [==============================] - 1s 12ms/step - loss: 0.2453 - accuracy: 0.7258
Epoch 5/100
49/49 [==============================] - 1s 13ms/step - loss: 0.2179 - accuracy: 0.7426
Epoch 6/100
49/49 [==============================] - 1s 13ms/step - loss: 0.1966 - accuracy: 0.7606
Epoch 7/100
49/49 [==============================] - 1s 14ms/step - loss: 0.1791 - accuracy: 0.7781
Epoch 8/100
49/49 [==============================] - 1s 14ms/step - loss: 0.1694 - accuracy: 0.7948
Epoch 9/100
49/49 

In [70]:
model = auto_model.export_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 920, 1)]          0         
                                                                 
 cast_to_float32 (CastToFloa  (None, 920, 1)           0         
 t32)                                                            
                                                                 
 conv1d (Conv1D)             (None, 914, 64)           512       
                                                                 
 conv1d_1 (Conv1D)           (None, 908, 32)           14368     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 151, 32)          0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 145, 32)           7200  

In [71]:
model.get_config()

{'name': 'model',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 920, 1),
    'dtype': 'float64',
    'sparse': False,
    'ragged': False,
    'name': 'input_1'},
   'name': 'input_1',
   'inbound_nodes': []},
  {'class_name': 'Custom>CastToFloat32',
   'config': {'name': 'cast_to_float32',
    'trainable': True,
    'dtype': 'float32'},
   'name': 'cast_to_float32',
   'inbound_nodes': [[['input_1', 0, 0, {}]]]},
  {'class_name': 'Conv1D',
   'config': {'name': 'conv1d',
    'trainable': True,
    'dtype': 'float32',
    'filters': 64,
    'kernel_size': (7,),
    'strides': (1,),
    'padding': 'valid',
    'data_format': 'channels_last',
    'dilation_rate': (1,),
    'groups': 1,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'ac

In [81]:
#model.save("model/mlc_cnn_autokeras_v2.4.h5")

INFO:tensorflow:Assets written to: model/mlc_cnn_autokeras_v2.4.pb\assets


# 3. Evaluation of the optimized model and saving the results

In [21]:
best_model = load_model("model/mlc_cnn_autokeras_v2.4.h5")

In [22]:
best_model.get_config()

{'name': 'model',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 920, 1),
    'dtype': 'float64',
    'sparse': False,
    'ragged': False,
    'name': 'input_1'},
   'name': 'input_1',
   'inbound_nodes': []},
  {'class_name': 'Custom>CastToFloat32',
   'config': {'name': 'cast_to_float32',
    'trainable': True,
    'dtype': 'float32'},
   'name': 'cast_to_float32',
   'inbound_nodes': [[['input_1', 0, 0, {}]]]},
  {'class_name': 'Conv1D',
   'config': {'name': 'conv1d',
    'trainable': True,
    'dtype': 'float32',
    'filters': 64,
    'kernel_size': (7,),
    'strides': (1,),
    'padding': 'valid',
    'data_format': 'channels_last',
    'dilation_rate': (1,),
    'groups': 1,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'ac

In [23]:
best_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 920, 1)]          0         
                                                                 
 cast_to_float32 (CastToFloa  (None, 920, 1)           0         
 t32)                                                            
                                                                 
 conv1d (Conv1D)             (None, 914, 64)           512       
                                                                 
 conv1d_1 (Conv1D)           (None, 908, 32)           14368     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 151, 32)          0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 145, 32)           7200  

In [24]:
prediction = best_model.predict(x_test)
y_pred = ch.make_pred_mlc(prediction, threshold=0.5)
acc, ham = ch.mlc_result(y_test, y_pred)



acc: 0.892
hamming loss: 0.0185

              precision    recall  f1-score   support

           0      1.000     1.000     1.000        50
           1      0.990     0.960     0.975       100
           2      0.959     0.940     0.949       200
           3      0.989     0.935     0.961       200
           4      0.986     0.710     0.826       100
           5      0.952     0.933     0.943       150
           6      0.987     0.987     0.987       150
           7      0.967     0.952     0.960       250

   micro avg      0.975     0.932     0.953      1200
   macro avg      0.979     0.927     0.950      1200
weighted avg      0.975     0.932     0.951      1200
 samples avg      0.959     0.937     0.942      1200



In [18]:
dfzip = ch.convert_and_zip_label(y_test, y_pred)
ch.cm_multi(dfzip)

Predicted,0,1,2,3,4,5,6,7,16,24,27,35,37,46,267,357,noclass,Total
True,,,,,,,,,,,,,,,,,,
0,50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50
1,0,46,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,50
2,0,0,44,0,0,3,0,0,0,0,0,0,0,1,0,0,2,50
3,0,0,0,48,0,0,0,0,0,0,0,1,0,0,0,0,1,50
4,0,0,3,0,39,1,0,1,0,5,0,0,0,0,0,0,1,50
5,0,0,0,0,0,49,0,0,0,0,0,1,0,0,0,0,0,50
6,0,0,0,0,0,0,50,0,0,0,0,0,0,0,0,0,0,50
7,0,0,0,0,0,0,0,47,0,0,0,0,0,0,0,1,2,50
16,0,2,0,0,0,0,0,0,48,0,0,0,0,0,0,0,0,50


In [19]:
res = ch.collect_results_mlc(y_test, y_pred, algorithm="cnn_autokeras", params="autokeras", best_param=None)
res

{'algorithm': 'cnn_autokeras',
 'params': 'autokeras',
 'best_param': None,
 'rec_0': 1.0,
 'pre_0': 1.0,
 'f1_0': 1.0,
 'rec_weighted': 0.9316666666666666,
 'pre_weighted': 0.9750229811748875,
 'f1_weighted': 0.9512837269654367,
 'acc': 0.892}

In [20]:
#ch.save_results_json(filename="mlc_algorithm_adaption_hyperopt_results_v2.4.json", 
                     key_algorithm = "cnn_autokeras", 
                     hyperopt_result = res)

existing file mlc_algorithm_adaption_hyperopt_results_v2.4.json opened and results of cnn_autokeras saved.
